In [ ]:
!pip install -U spacy-nightly --pre

     |████████████████████████████████| 12.7 MB 5.2 MB/s 
     |████████████████████████████████| 42 kB 1.3 MB/s 
     |████████████████████████████████| 457 kB 49.3 MB/s 
     |████████████████████████████████| 660 kB 64.0 MB/s 
     |████████████████████████████████| 9.1 MB 45.4 MB/s 
     |████████████████████████████████| 58 kB 5.6 MB/s 
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.2.0
    Uninstalling typing-extensions-4.2.0:
      Successfully uninstalled typing-extensions-4.2.0
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: smart-open
    Found existing installation: smart-open 6.0.0
    Uninstalling smart-open-6.0.0:
      Successfully uninstall

In [ ]:
!pip install -U pip setuptools wheel
!python -m spacy project clone tutorials/rel_component

     |████████████████████████████████| 2.1 MB 5.1 MB/s 
     |████████████████████████████████| 1.2 MB 51.3 MB/s 
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successfully uninstalled setuptools-57.4.0
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0+zzzcolab20220506162203 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.
spacy 2.2.4 requires catalogue<1.1.0,>=0.0.7, but you have catalogue 2.0.7 which is incompatible.
spacy 2.2.4 requires srsly<1.1.0,>=1.0.2, but you have srsly 2.4.3 which is incompatible.
spacy 2.2.4 requires thinc==7.4.0, but you have thinc 8.0.16 which is incompatible.
datascie

✔ Cloned 'tutorials/rel_component' from explosion/projects
/content/rel_component
✔ Your project is now ready!
To fetch the assets, run:
python -m spacy project assets /content/rel_component


In [ ]:
!python binary_converter.py

[None, None]
Traceback (most recent call last):
  File "binary_converter.py", line 123, in <module>
    main(ann, train_file, dev_file, test_file)
  File "binary_converter.py", line 70, in main
    doc.ents = entities
  File "spacy/tokens/doc.pyx", line 725, in spacy.tokens.doc.Doc.ents.__set__
  File "spacy/tokens/doc.pyx", line 1703, in spacy.tokens.doc.get_entity_info
TypeError: object of type 'NoneType' has no len()


In [ ]:
import json

# import typer
from pathlib import Path

from spacy.tokens import Span, DocBin, Doc
from spacy.vocab import Vocab
from wasabi import Printer
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English
from spacy.util import compile_infix_regex
import re
import spacy

nlp = spacy.blank("en")
# Create a blank Tokenizer with just the English vocab

msg = Printer()

SYMM_LABELS = ["Binds"]
MAP_LABELS = { "child_of" : "child_of", "king_of" : "king_of", "born_in" : "born_in", "master_in" : "master_in", "killed" : "killed", "lived_in" : "lived_in", "happened_in" : "happened_in", "spouse_of" : "spouse_of", 
"friend_of" : "friend_of", "leading" : "leading", "sibling_of" : "sibling_of", "is_a_part_of" : "is_a_part_of", "guardian_of" : "guardian_of", "killed" : "killed" }
# MAP_LABELS = {
#     "DEGREE_IN": "DEGREE_IN",
#     "EXPERIENCE_IN": "EXPERIENCE_IN"
# }
ann = "/content/testv3.txt"
# ann = "/content/relations_training.txt"
train_file='/content/rel_component/data/relations_training.spacy'
dev_file='/content/rel_component/data/relations_dev.spacy'
test_file='/content/rel_component/data/relations_test.spacy'

def main(json_loc: Path, train_file: Path, dev_file: Path, test_file: Path):
    """Creating the corpus from the Prodigy annotations."""
    Doc.set_extension("rel", default={},force=True)
    vocab = Vocab()

    docs = {"train": [], "dev": [], "test": [], "total": []}
    ids = {"train": set(), "dev": set(), "test": set(), "total":set()}
    count_all = {"train": 0, "dev": 0, "test": 0,"total": 0}
    count_pos = {"train": 0, "dev": 0, "test": 0,"total": 0}
    i = 0
    with open(json_loc, encoding="utf8") as jsonfile:
        file = json.load(jsonfile)
        for example in file:
            print(i)
            i+=1
            span_starts = set()
            neg = 0
            pos = 0
                    # Parse the tokens
            tokens=nlp(example["document"])    
            print(tokens)
            spaces=[]
            spaces = [True if tok.whitespace_ else False for tok in tokens]
            words = [t.text for t in tokens]
            doc = Doc(nlp.vocab, words=words, spaces=spaces)
            print(doc)

            # if(example['tokens'][0]['start'] == -1 or example['tokens'][1]['start'] == -1):
            #   continue
            
            if(example['tokens'][0]['start']<=example['tokens'][1]['start']):
              if(example['tokens'][0]['end']>example['tokens'][1]['start']):
                continue

            if(example['tokens'][1]['start']<=example['tokens'][0]['start']):
              if(example['tokens'][1]['end']>example['tokens'][0]['start']):
                continue
                      


            # Parse the GGP entities
            print('spans')
            spans = example["tokens"]
            print(spans)
            entities = []
            span_end_to_start = {}
            for span in spans:
                print(span)
                entity = doc.char_span(
                     span["start"], span["end"], label=span["entityLabel"]
                 )
                print(entity)
                if(entity == None):
                  x = 0
                  break
                # if(!type(entity)):
                #   continue
                span_end_to_start[span["token_start"]] = span["token_start"]
                #print(span_end_to_start)
                entities.append(entity)
                span_starts.add(span["token_start"])
            if(len(entities) != 2):
              continue
            print(entities)

            doc.ents = entities

            # Parse the relations
            rels = {}
            for x1 in span_starts:
                for x2 in span_starts:
                    rels[(x1, x2)] = {}
                    #print(rels)
            relations = example["relations"]
            #print(len(relations))
            for relation in relations:
                # the 'head' and 'child' annotations refer to the end token in the span
                # but we want the first token
                start = span_end_to_start[relation["head"]]
                end = span_end_to_start[relation["child"]]
                label = relation["relationLabel"]
                #print(rels[(start, end)])
                #print(label)
                #label = MAP_LABELS[label]
                if label not in rels[(start, end)]:
                    rels[(start, end)][label] = 1.0
                    pos += 1
                    #print(pos)
                    #print(rels[(start, end)])

            # The annotation is complete, so fill in zero's where the data is missing
            for x1 in span_starts:
                for x2 in span_starts:
                    for label in MAP_LABELS.values():
                        if label not in rels[(x1, x2)]:
                            neg += 1
                            rels[(x1, x2)][label] = 0.0

                            #print(rels[(x1, x2)])
            doc._.rel = rels
            #print(doc._.rel)

            # only keeping documents with at least 1 positive case
            if pos > 0:
                    docs["total"].append(doc)
                    count_pos["total"] += pos
                    count_all["total"] += pos + neg

                    
                    
    #print(len(docs["total"]))
    docbin = DocBin(docs=docs["total"], store_user_data=True)
    docbin.to_disk(test_file)
    msg.info(
        f"{len(docs['total'])} training sentences"
    )


In [ ]:
main(ann, train_file, dev_file, test_file)

0
This mighty-armed one, having fought Dhananjaya, the son of Pritha, and having reduced This mighty-armed one to great straits, was at last slain by This mighty-armed one antagonist.
This mighty-armed one, having fought Dhananjaya, the son of Pritha, and having reduced This mighty-armed one to great straits, was at last slain by This mighty-armed one antagonist.
spans
[{'text': 'having fought Dhananjaya', 'start': 23, 'end': 47, 'token_start': 4, 'token_end': 6, 'entityLabel': 'O'}, {'text': 'Pritha', 'start': 60, 'end': 66, 'token_start': 11, 'token_end': 11, 'entityLabel': 'O'}]
{'text': 'having fought Dhananjaya', 'start': 23, 'end': 47, 'token_start': 4, 'token_end': 6, 'entityLabel': 'O'}
having fought Dhananjaya
{'text': 'Pritha', 'start': 60, 'end': 66, 'token_start': 11, 'token_end': 11, 'entityLabel': 'O'}
Pritha
[having fought Dhananjaya, Pritha]
1
Behold, O Keshava, the two brothers of Avanti, Vinda and Anuvinda, lying there on the field, like two blossoming shala trees in 